# Raster to Vector Conversion in Pixel Art Images

Steps:

0) Import libraries
1) Load a pixel art image
2) Create an adjacency graph
3) Calculate and Render the Dual Graph
4) Simplify the Dual Graph
5) Rectify T Junctions
6) Interpolate and Render Splines
7) Smoothen the diagram

## 0) Import Libraries

In [1]:
import sys
sys.path.append('modules')

In [2]:
from IPython.display import SVG, display, HTML

from modules.pixel_adjacency_graph import PixelAdjacencyGraph
from modules.pixel_art_raster import PixelArtRaster
from modules.pixel_vector_graph import PixelVectorGraph
from modules.svg_renderer import SVGRenderer

In [3]:
import random
random.seed(69)

## 1) Load a Pixel Art Image

In [4]:
# TODO (P2): Add text explaining the variables

reduce_input_raster = False
save_reduced_input = False
render_scale_factor = 15
verbosity = 0

In [5]:
pixel_art_raster = PixelArtRaster(reduce_input_raster = True, svg_scale_factor = render_scale_factor, verbosity = verbosity)
pixel_art_raster.import_input_raster()

if save_reduced_input:
    pixel_art_raster.export_pixel_art_png()

In [6]:
pixel_art_raster.render()

## 2) Create an adjacency graph

<!-- The adjacency graph will be implemented as a 3D boolean adjacency matrix of shape $height \times width \times 8$ For the node $N$, the edges are denoted by the following indices

$$
\begin{pmatrix}
0 & 1 & 2 \\
3 & N & 4 \\
5 & 6 & 7
\end{pmatrix}
$$ -->

Two pixels can be marked as adjacent only if they share a corner or an edge, and if their colours are similar. Furthermore, the adjacency graph needs to be planar, as each connected component marks a continuous surface in the final vectorised image.

We start by defining an edge between each adjacent pixel, and pruning edges until the graph is planar. We forst prune edges between pixels that have dissimilar colours. Then, for every $2 \times 2$ complete subgraph, the 2 diagonal edges are pruned. This leaves overlapping diagonals only where the pixels form a $2 \times 2$ 'checkerboard-like' pattern, where it is not immediately clear which diagonal should be pruned.

To determine which edge is preserved and which is pruned, we check the following conditions, and apply them in decreasing order of priority.

1. If either of the edges is part of a chain of vertices of degree $2$, that edge is preserved.
2. If either of the edge colors is "sparse" compared to the other in an immediate $6 \times 6$ grid, the sparse pixels likely carry greater detail and are preserved.
3. If pruning an edge breaks the graph into more number of connected components, that edge is preserved. 

In [7]:
# The below graph takes the pixel art data and creates adjacency graph.
# By default, the class initialisation method automatically makes the graph planar
adjacency_graph = PixelAdjacencyGraph(pixel_art_raster)

In [8]:
adjacency_graph.render(svg_scale_factor = render_scale_factor)

# 3) Calculate and Render the Dual Graph

The dual graph here is equivalent to the *Voronoi diagram*. The plane of the adjacency graph is divided into regions, where each pixel has an area assigned to it. The area consists of all points that are closest to any of the pixel's half-edges, conpared to any other pixel's half-edge.

The Voronoi diagram visually shows each connected component of the adjacency graph as a continuous area. Essentially, the only remaining step is to adjust the boundary of each connected region to give it a smooth and natural appearance.

In [9]:
pixel_vector_graph = PixelVectorGraph(pixel_art_raster)
pixel_vector_graph.construct_dual_graph(adjacency_graph)

In [10]:
pixel_vector_graph.render(highlight_pixel_graph_edges=False, svg_scale_factor = render_scale_factor)

# 4) Simplify the Dual Graph

The Voronoi diagram above has a lot of boundary nodes that have a degree of $2$. These add unnecessary sharpness to the image. Removing these gives us a simplified image that give the diagram a visually smoother shape.

From here, we can essentially get a smooth looking diagram by simply replacing the straight edges with curves.

In [11]:
pixel_vector_graph.simplify_dual_graph()
pixel_vector_graph.render(svg_scale_factor = render_scale_factor)

# 5) Rectify T Junctions

In [12]:
pixel_vector_graph.resolve_t_junctions_in_simplified_vector_graph()
pixel_vector_graph.render(highlight_pixel_graph_edges=True, highlight_t_junction_edges=True, svg_scale_factor = render_scale_factor)

# 6) Interpolate and Render Splines

In [13]:
# TODO (P1): Instead of filling the gaps with triangles, it may be better to use De Casteljau subdivision.

pixel_vector_graph.render(show_smoothened_image = True, highlight_pixel_graph_edges=False, highlight_t_junction_edges=False, highlight_bezier_curves=False, svg_scale_factor = render_scale_factor)

# 7) Smoothen the diagram

The above gives a pretty good vectorised image, but may leave staircasing artifacts because of the jagged nature of diagonal lines in pixel art. We want to reduce these artifacts and essentially "smoothen" the image.

We can do so by offsetting the control points of the B-spline curves, optimising as per a loss function that attempts to balance increased smoothness with retaining the overall shape of the original image. For a node $i$, ihe loss function is as follows:

$$E^{(i)} = E_s^{(i)} + E_p^{(i)}$$

$E_s^{(i)}$ is a measure of smoothness of the image, which is measured by the curvature. Greater value of this function indicates greater curvature and thus lower smoothness. It is defined as follows:

$$E_s^{(i)} = \int_{s \in r(i)} |\kappa(s)| ds $$

where $r(i)$ is the set of all points that node $i$ has influence over. We approximate this by discrete uniform sampling along the curve.

$E_p^{(i)}$ is a measure of displacement. We want to allow small displacement with little penalty but have high penalty for huge movement. It is defined as follows:

$$E_p^{(i)} = ||p_i - \^{p_i}||^4$$

We optimise for the loss function iteratively. At each iteration, we traverse all nodes in a random order. For each node, we try several random (small) offset positions, and pick the one which lowers the loss function the most.

In [14]:
# TODO (P0): Lock all vertices that contribute to intentional sharpness of the image.

In [ ]:
# Perform iterative amoothening and render the smoothened image.

pixel_vector_graph.smoothen_vectorised_image(num_iterations=20, num_samples_per_iteration=10, sample_space_radius=0.01)
pixel_vector_graph.render(show_smoothened_image = True, highlight_pixel_graph_edges=False, highlight_t_junction_edges=False, highlight_bezier_curves=False, svg_scale_factor = render_scale_factor)